In [5]:
import numpy as np
import pandas as pd
import random
import Estimator as est
from Optimizer import OptVariance

np.set_printoptions(precision=3)

d = 10
dist_type = "uni"
est_type = "sw"
N=100000
total_repeat = 5
print("distribution=",dist_type, "estimitor=",est_type, 
      "total sample=",N, "total repeat num=",total_repeat)
x_max = 1
x_step_size = x_max/d
x_grid = x_step_size*np.arange(d)
print("x=", x_grid)

if dist_type=='exp':
    x_q = np.exp(-3*x_grid)
    x_q = x_q/np.sum(x_q)
elif dist_type=='uni':
    x_q = np.ones(d)/d
else:
    print('warning: invaild distribution type!')
print("q(x)=",x_q)

x_data = {'x': x_grid, 'q':x_q}
filename = "data/%s_q_%d.csv" %(dist_type,d)
pd.DataFrame(x_data).to_csv(filename)


idx_original = random.choices(np.arange(d),x_q,k=N)
sample_original = x_grid[idx_original]
print('%d data samples generated with %s distribution.\n' %(N, dist_type))

eps_grid = 0.5*np.arange(1,6)
for i in range(len(eps_grid)):
    eps= eps_grid[i]
    print('eps=%.2f'%(eps))

    if est_type == 'sw':
        a_grid, M = est.SquareWave(eps,x_grid)
    elif est_type == 'grr':
        a_grid, M = est.GenRandResp(eps, x_grid)
    else:
        print('warning: invalid estimator type!')
    print('Found matrix M for %s estimator!' %(est_type))

    filename = 'data/%s_M_%.2f_%d.csv'%(est_type,eps,d)
    pd.DataFrame(M).to_csv(filename)
    filename = 'data/%s_a_%.2f_%d.csv'%(est_type,eps,d)
    pd.DataFrame(a_grid).to_csv(filename)

    rand_num = np.zeros((N,d))
    for j in range(d):
        rand_num[:,j] = random.choices(a_grid,M[:,j],k=N)
    print("Random num pools generated.")

    x_q_noisy = np.zeros((1,d))
    for k in range(total_repeat):
        idx_noise = random.choices(range(N),k=N)
        idx_pair = list(zip(idx_noise,idx_original))
        sample_perturbed = rand_num[tuple(zip(*idx_pair))]
        elements,counts = np.unique(sample_perturbed,return_counts=True)
        x_q_est = est.EM(M,counts,eps)
        x_q_noisy +=x_q_est
    x_q_noisy = x_q_noisy/np.sum(x_q_noisy)
    print('Distributed Estimation with %s complete with %d repeats' 
        %(est_type, total_repeat))
    print('q_est(x)=', x_q_noisy)

    # Find the AAA transition matrix
    _, M = OptVariance(eps, x_grid, x_q_noisy)
    filename = 'data/%s_%s_M_%.2f_%d.csv'%(est_type,dist_type,eps,d)
    pd.DataFrame(M).to_csv(filename)
    print("AAA Solution Found!\n")

distribution= uni estimitor= sw total sample= 100000 total repeat num= 5
x= [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
q(x)= [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
100000 data samples generated with uni distribution.

eps=0.50
Found matrix M for sw estimator!
Random num pools generated.
Distributed Estimation with sw complete with 5 repeats
q_est(x)= [[0.088 0.115 0.091 0.105 0.103 0.108 0.095 0.109 0.092 0.094]]
AAA Solution Found!

eps=1.00
Found matrix M for sw estimator!
Random num pools generated.
Distributed Estimation with sw complete with 5 repeats
q_est(x)= [[0.102 0.096 0.105 0.097 0.097 0.103 0.103 0.093 0.103 0.1  ]]
AAA Solution Found!

eps=1.50
Found matrix M for sw estimator!
Random num pools generated.
Distributed Estimation with sw complete with 5 repeats
q_est(x)= [[0.099 0.1   0.101 0.099 0.103 0.099 0.103 0.098 0.1   0.1  ]]
AAA Solution Found!

eps=2.00
Found matrix M for sw estimator!
Random num pools generated.
Distributed Estimation with sw complete with 5 rep